In [1]:
import os

os.system("pip3 install --upgrade pip")

os.system("python3 -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple")

os.system("pip install Shapely-1.8.2-cp311-cp311-win_amd64.whl")
os.system("pip install Shapely-1.8.1.post1-cp37-cp37m-win_amd64.whl")
os.system("pip install \"paddleocr>=2.0.1\"")


0

In [2]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image
from PIL import ImageDraw

In [3]:
ocr = PaddleOCR(use_angle_cls=True, lang='ch') # need to run only once to download and load model intoTkinter memory
target_dir = ['南京银行录音设备', '宁波影像设备采购', '上海市分行验印摄像仪', '深圳分行验印', '信用合作清算中心验印', '亚太财险']
img_filename = ['1', '2', '3', '4', '5']
# target_dir = ['南京银行录音设备']
# img_filename = ['2']

[2022/10/03 16:16:39] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, shape_info_filename=None, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\LoveForKnowledge/.paddleocr/whl\\det\\ch\\ch_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\LoveForKnowledge/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv3_rec_infer', rec_image_shape='3,

In [4]:
def print_diff(i, origin, origin_text, scan, scan_text):
    print(f'file: {origin}')
    print(f'data: {origin_text}')
    print(f'file: {scan}')
    print(f'data: {scan_text}')
    print(f'i: {i}')
    print()

In [5]:
def compare(origin, scan, save):
    origin_result = ocr.ocr(origin, cls=True)
    scan_result = ocr.ocr(scan, cls=True)

    # 去除空格    
    for xx in origin_result:
        xx[1] = str.replace(xx[1][0], ' ', '')
    for xx in scan_result:
        xx[1] = str.replace(xx[1][0], ' ', '')


    # 识别内容->存储数组的index 的映射.
    scan_set = {}
    origin_set = {}
    for i in range(len(origin_result)):
        if origin_result[i][1] in origin_set:
            origin_set[origin_result[i][1]].append(i)
        else:
            origin_set[origin_result[i][1]] = [i]
    for i in range(len(scan_result)):
        if scan_result[i][1] in scan_set:
            scan_set[scan_result[i][1]].append(i)
        else:
            scan_set[scan_result[i][1]] = [i]


    diff_pos_org = []
    diff_pos_scan = []
    # 检查是否有对应的相同字符
    for xx in origin_result:
        if xx[1] not in scan_set:
            diff_pos_org.append(xx[0])
            print(xx)
        else:
            # 位置差异太大也要标注出来
            mn = 1000
            for i in scan_set[xx[1]]:
                yy = scan_result[i]
                mn = min(mn, abs(xx[0][0][0] - yy[0][0][0]) + abs(xx[0][0][1] - yy[0][0][1]))
            if mn > 100:
                diff_pos_org.append(xx[0])
    for xx in scan_result:
        if xx[1] not in origin_set:
            diff_pos_scan.append(xx[0])
            print(xx)
        else:
            # 位置差异太大也要标注出来
            mn = 1000
            for i in origin_set[xx[1]]:
                yy = origin_result[i]
                mn = min(mn, abs(xx[0][0][0] - yy[0][0][0]) + abs(xx[0][0][1] - yy[0][0][1]))
            if mn > 100:
                diff_pos_scan.append(xx[0])

    # 打印内容
    img_origin = Image.open(origin)
    img_draw_origin = ImageDraw.ImageDraw(img_origin)
    img_scan = Image.open(scan)
    img_draw_scan = ImageDraw.ImageDraw(img_scan)

    for diff in diff_pos_scan:
        img_draw_scan.rectangle((diff[0][0], diff[0][1], diff[2][0], diff[2][1]), fill=None, outline='red', width=3)
    for diff in diff_pos_org:
        img_draw_origin.rectangle((diff[0][0], diff[0][1], diff[2][0], diff[2][1]), fill=None, outline='red', width=3)

    result = Image.new("RGB", (img_origin.size[0] + img_scan.size[0], img_origin.size[1]))
    loc1, loc2 = (0, 0), (img_origin.size[0], 0)
    result.paste(img_origin, loc1)
    result.paste(img_scan, loc2)
    result.save(save + 'result.jpg')

In [6]:
# origin = './img/' + target_dir[0] + "/origin/2.jpg"
# scan = './img/' + target_dir[0] + "/scan/2.JPG"
# compare(origin, scan)

In [7]:
# result = ocr.ocr(img_path, cls=True)
# print(result)
# for line in result:
#     print(line)

In [8]:
for t_dir in target_dir:
    for i_name in img_filename:
        origin = './img/' + t_dir + '/origin/' + i_name + '.jpg'
        scan = './img/' + t_dir + '/scan/' + i_name + '.JPG'
        save = './img/' + t_dir + '/' + i_name
        compare(origin, scan, save)

[2022/10/03 16:16:44] ppocr DEBUG: dt_boxes num : 5, elapse : 1.6212737560272217
[2022/10/03 16:16:44] ppocr DEBUG: cls num  : 5, elapse : 0.11297225952148438
[2022/10/03 16:16:47] ppocr DEBUG: rec_res num  : 5, elapse : 2.6023123264312744
[2022/10/03 16:16:48] ppocr DEBUG: dt_boxes num : 5, elapse : 1.1555280685424805
[2022/10/03 16:16:48] ppocr DEBUG: cls num  : 5, elapse : 0.09533500671386719
[2022/10/03 16:16:50] ppocr DEBUG: rec_res num  : 5, elapse : 2.2199740409851074
[2022/10/03 16:16:51] ppocr DEBUG: dt_boxes num : 43, elapse : 1.106945276260376
[2022/10/03 16:16:52] ppocr DEBUG: cls num  : 43, elapse : 0.7731742858886719
[2022/10/03 16:17:13] ppocr DEBUG: rec_res num  : 43, elapse : 20.5243182182312
[2022/10/03 16:17:14] ppocr DEBUG: dt_boxes num : 44, elapse : 1.3154196739196777
[2022/10/03 16:17:15] ppocr DEBUG: cls num  : 44, elapse : 0.8205969333648682
[2022/10/03 16:17:36] ppocr DEBUG: rec_res num  : 44, elapse : 20.699395895004272
[[[347.0, 312.0], [709.0, 312.0], [709.

--------
以下为测试用

In [ ]:
origin_result = ocr.ocr(origin, cls=True)
scan_result = ocr.ocr(scan, cls=True)

# 去除空格    
for xx in origin_result:
    xx[1] = str.replace(xx[1][0], ' ', '')
for xx in scan_result:
    xx[1] = str.replace(xx[1][0], ' ', '')



In [ ]:
# 识别内容->存储数组的index 的映射.
scan_set = {}
origin_set = {}
for i in range(len(origin_result)):
    if origin_result[i][1] in origin_set:
        origin_set[origin_result[i][1]].append(i)
    else:
        origin_set[origin_result[i][1]] = [i]
for i in range(len(scan_result)):
    if scan_result[i][1] in scan_set:
        scan_set[scan_result[i][1]].append(i)
    else:
        scan_set[scan_result[i][1]] = [i]


diff_pos_org = []
diff_pos_scan = []
# 检查是否有对应的相同字符
for xx in origin_result:
    if xx[1] not in scan_set:
        diff_pos_org.append(xx[0])
        print(xx[1])
    else:
        # 位置差异太大也要标注出来
        mn = 1000
        for i in scan_set[xx[1]]:
            yy = scan_result[i]
            mn = min(mn, abs(xx[0][0][0] - yy[0][0][0]) + abs(xx[0][0][1] - yy[0][0][1]))
        if mn > 80:
            diff_pos_org.append(xx[0])
            print(xx, yy)
for xx in scan_result:
    if xx[1] not in origin_set:
        diff_pos_scan.append(xx[0])
    else:
        # 位置差异太大也要标注出来
        mn = 1000
        for i in origin_set[xx[1]]:
            yy = origin_result[i]
            mn = min(mn, abs(xx[0][0][0] - yy[0][0][0]) + abs(xx[0][0][1] - yy[0][0][1]))
        if mn > 80:
            diff_pos_scan.append(xx[0])
            print(xx[1], xx[0])

# 打印内容
img_origin = Image.open(origin)
img_draw_origin = ImageDraw.ImageDraw(img_origin)
img_scan = Image.open(scan)
img_draw_scan = ImageDraw.ImageDraw(img_scan)

for diff in diff_pos_scan:
    img_draw_scan.rectangle((diff[0][0], diff[0][1], diff[2][0], diff[2][1]), fill=None, outline='red', width=5)
for diff in diff_pos_org:
    img_draw_origin.rectangle((diff[0][0], diff[0][1], diff[2][0], diff[2][1]), fill=None, outline='red', width=5)

result = Image.new("RGB", (img_origin.size[0] + img_scan.size[0], img_origin.size[1]))
loc1, loc2 = (0, 0), (img_origin.size[0], 0)
result.paste(img_origin, loc1)
result.paste(img_scan, loc2)
result.save(save + 'result.jpg')

址：南京市中山路288号
[第一条]］购销内容


In [ ]:
# draw result
# image = Image.open(img_path).convert('RGB')
# boxes = [line[0] for line in result]
# txts = [line[1][0] for line in result]
# scores = [line[1][1] for line in result]
# im_show = draw_ocr(image, boxes, txts, scores, font_path='./fonts/simfang.ttf')
# im_show = Image.fromarray(im_show)
# im_show.save('./img/result.jpg')